In [101]:
#Code proposé par M.Nataf:Séance 1
import scipy.io as spi
import numpy as np
import matplotlib.pyplot as plt

In [102]:
mat=spi.loadmat("mnist-original.mat")
data=np.transpose(mat['data'])
label=np.array(mat['label']) #label: chiffre numérisé
label=label.astype(int) #Les labels sont stockés en flottants, on les convertit en entiers

Découpe aléatoire de la base de données en base d'apprentissage et base de test

In [142]:
Y,y=data,label[0]
#On change l'ordre des données et des labels avec la même permutation pour que data_test et data_app soient hétérogènes...
def decoupe_aleatoire_base(Y,y):
    m=np.random.permutation((len(y))) #permutation arbitraire
    Y_m=Y[m] #data après permutation
    y_m=y[m] #labels après permutation
    n=len(Y) #nombre d'images dans la base donnée
    n_80=80*n/100 #nombre d'images dans la base d'apprentissage
    n_80=int(n_80)
    data_app=Y_m[:n_80] #base d'apprentissage
    label_app=y_m[:n_80]
    data_test=Y_m[n_80:] #base de tests
    label_test=y_m[n_80:]
    return data_app,label_app,data_test,label_test

Dans la base d'apprentissage, calcul des centoides des chiffres

In [143]:
def calcul_centroides(data_app,label_app):
    X,x=data_app,label_app 
    moy_chiff=[]
    for i in range(10):
        moy=np.mean(X[x==i],axis=0)
        moy_chiff+=[moy]
    return moy_chiff

Méthode d'estimation par rapport à la distance euclidienne

In [144]:
#Définition de la fonction qui estime le chiffre d'un vecteur de la base de tests:
def estim_chiffre_1(v,moy_chiff):
    distances=np.array([np.linalg.norm(v-u) for u in moy_chiff]) #distances entre v et les "chiffres moyens"
    return np.argmin(distances)

Méthode d'estimation par rapport à la "distance" cosine

In [145]:
#On définit la fonction cosine:
def cosine(u,v):
    return np.inner(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))

In [146]:
#Fonction qui estime le chiffre d'un vecteur de la base de tests vis a vis de la "distance" cosine:
def estim_chiffre_2(v,moy_chiff):
    distances=np.array([cosine(u,v) for u in moy_chiff]) #distances cosine entre v et les "chiffres moyens"
    return np.argmax(distances) #On prend le max car plus l'angle est petit, plus le cos est grand

Méthode d'estimation par rapport à la norme p

In [147]:
#Fonction qui estime le chiffre d'un vecteur de la base de tests vis a vis de la norme p:
def estim_chiffre_p(p,v,moy_chiff):
    distances=np.array([np.linalg.norm(v-u,p) for u in moy_chiff]) #distances entre v et les "chiffres moyens" en norme p
    return np.argmin(distances)

Calcul du pourcentage de prédictions correctes sur la base de tests

In [149]:
#Labels estimés pour les vecteurs de la base de tests et pour une certaine distance:
def precision_estimation_label(data_test,label_test,methode,p,moy_chiff): # renseigner un p quelconque si on n'utilise pas la norme p (par ex p=0)
    k=len(data_test)
    label_estim=np.zeros(k)
        
    if (methode != estim_chiffre_p):      
        for i in range(k):
            label_estim[i]=methode(data_test[i],moy_chiff)
            #print("la précision obtenue est de", np.mean(label_estim==label_test))
            
    else:
        for i in range(k):
            label_estim[i]=methode(p, data_test[i],moy_chiff)
            #print("la précision obtenue pour p={} est de".format(p), np.mean(label_estim==label_test))
        
    return np.mean(label_estim==label_test) #  pourcentage de prédictions correctes sur la base de tests.

Fonction permettant de calculer la précision d une méthode (pour une seule découpe initiale)

In [171]:
def Precision_methode(methode,p,Y,y):    
    data_app,label_app,data_test,label_test=decoupe_aleatoire_base(Y,y)
    moy_chiff=calcul_centroides(data_app,label_app)
    precision=precision_estimation_label(data_test,label_test,methode,p,moy_chiff)
    return precision

CALCUL DE LA MOYENNE ET DE LA VARIANCE DE NOS METHODES DE PREDICTION POUR PLUSIEURS DECOUPES ALEATOIRES DE LA BDD

In [172]:
def Moyenne_variance(iterations,methode,p,Y,y):
    Predictions=[]
    
    for i in range(iterations):
        precision=Precision_methode(methode,p,Y,y)
        Predictions.append(precision)
    
    Predictions=np.array(Predictions)
    Moyenne=np.mean(Predictions)
    Variance=np.var(Predictions)
    print("Moyenne:",Moyenne)
    print("Variance:", Variance)
    return Moyenne, Variance

Moyenne et variance de la méthode de la norme euclidienne pour 100 découpes aléatoires de la BDD

In [170]:
Moyenne_variance(100,estim_chiffre_1,0,Y,y)

Moyenne: 0.8094549999999999
Variance: 1.0116903571428555e-05


(0.8094549999999999, 1.0116903571428555e-05)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 80,6%).

Moyenne et variance de la méthode cosine pour 100 découpes aléatoires de la BDD

In [162]:
Moyenne_variance(100,estim_chiffre_2,0,Y,y)

Moyenne: 0.8146835714285714
Variance: 6.854413775510206e-06


(0.8146835714285714, 6.854413775510206e-06)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 81,4%).

Moyenne et variance de la méthode de la norme p pour 100 découpes aléatoires de la BDD

Avec p=3

In [163]:
Moyenne_variance(100,estim_chiffre_p,3,Y,y)

Moyenne: 0.820355
Variance: 8.882903571428592e-06


(0.820355, 8.882903571428592e-06)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 82,05%).

Avec p=4

In [164]:
Moyenne_variance(100,estim_chiffre_p,4,Y,y)

Moyenne: 0.8115399999999999
Variance: 9.961461224489778e-06


(0.8115399999999999, 9.961461224489778e-06)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 81,5%).

Avec p=5

In [165]:
Moyenne_variance(100,estim_chiffre_p,5,Y,y)

Moyenne: 0.8000042857142857
Variance: 6.213655102040815e-06


(0.8000042857142857, 6.213655102040815e-06)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 80,5%).

Avec p=6

In [167]:
Moyenne_variance(100,estim_chiffre_p,6,Y,y)

Moyenne: 0.7898971428571427
Variance: 1.1379216326530578e-05


(0.7898971428571427, 1.1379216326530578e-05)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 79,5%).

Avec p=7

In [173]:
Moyenne_variance(100,estim_chiffre_p,7,Y,y)

Moyenne: 0.7809178571428572
Variance: 1.0354221938775518e-05


(0.7809178571428572, 1.0354221938775518e-05)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 78,6%).

Avec p=8

In [174]:
Moyenne_variance(100,estim_chiffre_p,8,Y,y)

Moyenne: 0.7728050000000001
Variance: 1.1948454591836782e-05


(0.7728050000000001, 1.1948454591836782e-05)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 77,9%).

Avec p=9

In [175]:
Moyenne_variance(100,estim_chiffre_p,9,Y,y)

Moyenne: 0.7673378571428571
Variance: 1.208763826530608e-05


(0.7673378571428571, 1.208763826530608e-05)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 77,2%).

Avec p=10

In [176]:
Moyenne_variance(100,estim_chiffre_p,10,Y,y)

Moyenne: 0.7607971428571428
Variance: 8.618848979591805e-06


(0.7607971428571428, 8.618848979591805e-06)

(Pour rappel, notre estimation de précision pour une seule découpe aléatoire initiale était de 76,5%).

---------------------------------------------------------------------------------------------

Conclusion

Pour 100 itérations de découpe aléatoire de la base de donnée, on constate que la variance obtenue pour chacune des méthodes est de l'ordre de 10^(-5) ou de 10^(-6) et est donc négligeable. 

Les estimations de précision obtenues en prenant la moyenne de 100 découpes de la base différentes donnent toutes à moins de 0.6% de différence près les mêmes résultats que nos premières estimations avec une seule découpe aléatoire initiale. 

Le code développé dans cette sous-section, censé "gommer" l'impact de l'aléatoire dans la découpe initiale de notre base de donnée sur l'estimation de la précision de nos méthodes semble donc d'un intérêt limité. 
L'impact négligeable de l'aléatoire dans la découpe de notre base de données peut être du à la taille limitée de celle-ci (70.000 entrées de chiffres manuscrits) ou encore à l'optimisation de la fonction "random.permutation" de Python que nous utilisons dans notre code.

En conclusion: au vu de l'impact négligeable de l'aléatoire dans la découpe de notre base de données pour cette première méthode étudiée, on considérera comme satisfaisante la-dite méthode de découpe. Nous nous contenterons donc d'une seule découpe aléatoire initiale dans la suite de cette section. 

------------------------------------------